In [1]:
!pip install stable-baselines3[extra]

In [2]:
try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

Cloning into 'boptestGymService'...


In [3]:
import requests
import sys
import torch
import gymnasium as gym
import os

In [4]:
sys.path.insert(0,'boptestGymService')

url = 'https://api.boptest.net'

In [5]:
from stable_baselines3 import TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from boptestGymService.boptestGymEnv import BoptestGymEnv
from boptestGymEnv import DiscretizedActionWrapper
from gym.wrappers import FlattenObservation
from boptestGymEnv import DiscretizedObservationWrapper

C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:122: SyntaxWarning: invalid escape sequence '\c'
  axs[0].set_ylabel('Operative\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:131: SyntaxWarning: invalid escape sequence '\c'
  axs[3].set_ylabel('Ambient\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:135: SyntaxWarning: invalid escape sequence '\d'
  axt.plot(x_time, df['weaSta_reaWeaHDirNor_y'], color='gold', linestyle='-', linewidth=1, label='$\dot{Q}_rad$')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:141: SyntaxWarning: invalid escape sequence '\d'
  axs[3].plot([],[], color='gold',        linestyle='-', linewidth=1, label='$\dot{Q}_{rad}$')


In [6]:
# Redefine reward function
class BoptestGymEnvCustomReward(BoptestGymEnv):
    '''Define a custom reward for this building

    '''
    def get_reward(self):
        '''Custom reward function. To expedite learning, we use a clipped reward
        function that has a value of 1 when there is no increase in discomfort
        and 0 otherwise. We use the BOPTEST `GET /kpis` API call to compute the
        total cummulative discomfort from the beginning of the episode. Note
        that this is the true value that BOPTEST uses when evaluating
        controllers.

        '''
        # Compute BOPTEST core kpis
        kpis = requests.get('{0}/kpi/{1}'.format(self.url, self.testid)).json()['payload']
        # Calculate objective integrand function as the total discomfort
        objective_integrand = kpis['tdis_tot']
        # Give reward if there is not immediate increment in discomfort
        if objective_integrand == self.objective_integrand:
          reward=1
        else:
          reward=0
        # Record current objective integrand for next evaluation
        self.objective_integrand = objective_integrand
        return reward

##KPIs##

Thermal discomfort: reported with units of [ Kh/zone ], defines the cumulative deviation of zone temperatures from upper and lower comfort limits that are predefined within the test case FMU for each zone, averaged over all zones. Air temperature is used for air-based systems and operative temperature is used for radiant systems.

Indoor Air Quality (IAQ) Discomfort: reported with units of [ ppmh/zone ], defines the extent that the CO 2  concentration levels in zones exceed bounds of the acceptable concentration level, which are predefined within the test case FMU for each zone, averaged over all zones.

Energy Use: reported with units of [ kWh/m2 ], defines the HVAC energy usage.

Peak electrical demand:reported with units of [ kW/m2 ], defines the HVAC peak electrical demand.

Peak gas demand:reported with units of [ kW/m2 ], defines the HVAC peak gas demand.

Peak district heating demand:reported with units of [ kW/m2 ], defines the HVAC peak district heating demand.

Cost: reported with units of [USD/ m2 ] or [EUR/ m2 ], defines the operational cost associated with the HVAC energy usage.

Emissions: reported with units of [ kgCO2/m2 ], defines the  CO2  emissions from the HVAC energy usage.

Computational time ratio: defines the average ratio between the controller computation time and the test simulation control step. The controller computation time is measured as the time between two emulator advances.

In [11]:
test_case="multizone_office_simple_air"

In [15]:
BoptestGymEnv?

Init signature:
BoptestGymEnv(
    url='http://127.0.0.1',
    testcase='bestest_hydronic_heat_pump',
    actions=['oveHeaPumY_u'],
    observations={'reaTZon_y': (280.0, 310.0)},
    reward=['reward'],
    max_episode_length=10800,
    random_start_time=False,
    excluding_periods=None,
    regressive_period=None,
    predictive_period=None,
    start_time=0,
    warmup_period=0,
    scenario={'electricity_price': 'constant'},
    step_period=900,
    render_episodes=False,
    log_dir='C:\\Users\\irmak\\Documents\\GitHub\\RL_ex',
)
Docstring:     
BOPTEST Environment that follows gym interface.
This environment allows the interaction of RL agents with building
emulator models from BOPTEST. 
 
Init docstring:
Parameters
----------
url: string
    Rest API url for communication with the BOPTEST interface
testcase: string
    The string identifier of the testcase
actions: list
    List of strings indicating the action space. The bounds of 
    each variable from the action space the ar

In [17]:
env = BoptestGymEnvCustomReward(url                   = url,
                    testcase              = test_case,
                    actions               = ['hvac_oveZonSupCor_TZonCooSet_u', 
                                             'hvac_oveZonSupEas_TZonCooSet_u', 
                                             'hvac_oveZonSupNor_TZonCooSet_u', 
                                             'hvac_oveZonSupWes_TZonCooSet_u',
                                             'hvac_oveZonSupSou_TZonCooSet_u'],
                    observations          = {"hvac_reaZonCor_TZon_y":(0, 35),
                                             "hvac_reaZonEas_TZon_y":(0, 35),
                                             "hvac_reaZonWes_TZon_y":(0, 35),
                                             "hvac_reaZonNor_TZon_y":(0, 35),
                                             "hvac_reaZonSou_TZon_y":(0, 35)},
                    random_start_time     = False,
                    start_time            = 31*24*3600,
                    max_episode_length    = 24*3600,
                    warmup_period         = 0,
                    step_period           = 900,
                    predictive_period     =10 )

C:\Users\irmak\anaconda3\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
C:\Users\irmak\anaconda3\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [19]:
log_path =os.path.join("Training", "Boptest", "Logs")
TD3_path = os.path.join("Training" , "Saved Models", "TD3_Boptest")

In [21]:
TD3?

Init signature:
TD3(
    policy: Union[str, type[stable_baselines3.td3.policies.TD3Policy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.001,
    buffer_size: int = 1000000,
    learning_starts: int = 100,
    batch_size: int = 256,
    tau: float = 0.005,
    gamma: float = 0.99,
    train_freq: Union[int, tuple[int, str]] = 1,
    gradient_steps: int = 1,
    action_noise: Optional[stable_baselines3.common.noise.ActionNoise] = None,
    replay_buffer_class: Optional[type[stable_baselines3.common.buffers.ReplayBuffer]] = None,
    replay_buffer_kwargs: Optional[dict[str, Any]] = None,
    optimize_memory_usage: bool = False,
    policy_delay: int = 2,
    target_policy_noise: float = 0.2,
    target_noise_clip: float = 0.5,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[dict[str, Any]] = None,
    verbose: int = 0,
    seed: Optional[int] = None,
 

In [23]:
model = TD3('MlpPolicy', env, verbose = 1, tensorboard_log = log_path, learning_rate=0.001, gamma= 0.99, batch_size= 100, policy_delay=2)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [27]:
model.learn(total_timesteps=1000)

Logging to Training\Boptest\Logs\TD3_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 96       |
|    ep_rew_mean     | 59.8     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 0        |
|    time_elapsed    | 769      |
|    total_timesteps | 384      |
| train/             |          |
|    actor_loss      | 14.1     |
|    critic_loss     | 0.443    |
|    learning_rate   | 0.001    |
|    n_updates       | 283      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 96       |
|    ep_rew_mean     | 62.4     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 0        |
|    time_elapsed    | 1479     |
|    total_timesteps | 768      |
| train/             |          |
|    actor_loss      | 13.4     |
|    critic_loss     | 0.555    |
|    learning_rate   | 0.001    |
|    n_up

In [29]:
model.save(TD3_path)

In [31]:
evaluate_policy(model, env, n_eval_episodes=3)

C:\Users\irmak\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(65.0, 0.0)

In [33]:
env.get_kpis()

{'tdis_tot': 0,
 'idis_tot': 0,
 'ener_tot': 0,
 'cost_tot': 0,
 'emis_tot': 0,
 'pele_tot': 4.720136809681116e-12,
 'pgas_tot': None,
 'pdih_tot': None,
 'time_rat': None}